# 準備

## artのインストール

In [ ]:
!pip3 install adversarial-robustness-toolbox

## ライブラリのインポート

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [ ]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR10のラベル。
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

In [ ]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## 摂動を作るためのClassifierの作成



### モデル定義

In [ ]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model = Model(inputs, y)

# モデルのコンパイル。
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 16, 128)       1475

### 学習の実行

In [ ]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - ETA: 0s - loss: 1.9355 - accuracy: 0.2498

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 32s 644us/sample - loss: 1.9355 - accuracy: 0.2498 - val_loss: 1.8009 - val_accuracy: 0.3229
Epoch 2/30
50000/50000 [==============================] - 32s 632us/sample - loss: 1.6090 - accuracy: 0.3832 - val_loss: 1.4905 - val_accuracy: 0.4470
Epoch 3/30
50000/50000 [==============================] - 32s 631us/sample - loss: 1.3596 - accuracy: 0.4912 - val_loss: 1.3059 - val_accuracy: 0.5337
Epoch 4/30
50000/50000 [==============================] - 32s 632us/sample - loss: 1.1825 - accuracy: 0.5657 - val_loss: 1.3198 - val_accuracy: 0.5198
Epoch 5/30
50000/50000 [==============================] - 31s 630us/sample - loss: 1.0590 - accuracy: 0.6137 - val_loss: 1.0861 - val_accuracy: 0.6161
Epoch 6/30
50000/50000 [==============================] - 32s 632us/sample - loss: 0.9485 - accuracy: 0.6591 - val_loss: 0.9956 - val_accuracy: 0.6518
Epoch 7/30
50000/50000 [==============================] - 32s 631us/sample - loss: 0.8488 - accuracy: 0.6

### モデルの精度評価

In [ ]:
model.evaluate(X_test, y_test)

[0.8451863438129426, 0.814]

## Assistive Signalsの作成

In [ ]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

In [ ]:
# FGSMインスタンスの作成。
attack_targeted = FastGradientMethod(estimator=classifier, eps=0.10, targeted=True)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_test_assistive = attack_targeted.generate(x=X_test, y=y_test)

## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [ ]:
attack_notargeted = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
X_test_adv = attack_notargeted.generate(X_test)

## 敵対的画像に対するモデルの精度評価

### cleanのみで学習したモデルとAssistive Signals

In [ ]:
model.evaluate(X_test_assistive, y_test)

[0.06220840414933918, 0.9837]

### cleanのみで学習したモデルと敵対的攻撃画像

In [ ]:
model.evaluate(X_test_adv, y_test)

[14.784507040405273, 0.1109]

# Assistive Signals入りの画像のみで学習

In [ ]:
X_train_assistive = attack_targeted.generate(x=X_train, y=y_train)

In [ ]:
model2 = Model(inputs, y)

# モデルのコンパイル。
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.fit(X_train_assistive, y_train,
          batch_size=512,
          epochs=30,
          shuffle=True)

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 31s 614us/sample - loss: 0.1384 - accuracy: 0.9650
Epoch 2/30
50000/50000 [==============================] - 31s 612us/sample - loss: 0.0163 - accuracy: 0.9945
Epoch 3/30
50000/50000 [==============================] - 31s 610us/sample - loss: 0.0095 - accuracy: 0.9970
Epoch 4/30
50000/50000 [==============================] - 31s 612us/sample - loss: 0.0057 - accuracy: 0.9982
Epoch 5/30
50000/50000 [==============================] - 31s 610us/sample - loss: 0.0078 - accuracy: 0.9972
Epoch 6/30
50000/50000 [==============================] - 31s 611us/sample - loss: 0.0087 - accuracy: 0.9972
Epoch 7/30
50000/50000 [==============================] - 31s 610us/sample - loss: 0.0042 - accuracy: 0.9986
Epoch 8/30
50000/50000 [==============================] - 31s 612us/sample - loss: 0.0039 - accuracy: 0.9988
Epoch 9/30
50000/50000 [==============================] - 31s 611us/sample - loss: 0.0048 - accuracy: 0.9

In [ ]:
# assistive train data
model2.evaluate(X_train_assistive, y_train)

[0.012166566797296401, 0.99586]

In [ ]:
# clean test data
model2.evaluate(X_test, y_test)

[0.8451863438129426, 0.814]

In [ ]:
# assistive test data
model2.evaluate(X_test_assistive, y_test)

[0.06220840414933918, 0.9837]

In [ ]:
# adversarial test data
model2.evaluate(X_test_adv, y_test)

[14.784507040405273, 0.1109]

# Assistive Signals入り画像とクリーン画像を5:5で学習

In [ ]:
X_train_half = list(X_train[:25000])
X_train_half.extend(X_train_assistive[25000:])
X_train_half = np.array(X_train_half)

In [ ]:
model3 = Model(inputs, y)

# モデルのコンパイル。
model3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model3.fit(X_train_half, y_train,
          batch_size=512,
          epochs=30,
          shuffle=True)

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 30s 602us/sample - loss: 0.2670 - accuracy: 0.9125
Epoch 2/30
50000/50000 [==============================] - 30s 603us/sample - loss: 0.1458 - accuracy: 0.9479
Epoch 3/30
50000/50000 [==============================] - 30s 603us/sample - loss: 0.1096 - accuracy: 0.9614
Epoch 4/30
50000/50000 [==============================] - 30s 602us/sample - loss: 0.0849 - accuracy: 0.9699
Epoch 5/30
50000/50000 [==============================] - 30s 601us/sample - loss: 0.0763 - accuracy: 0.9724
Epoch 6/30
50000/50000 [==============================] - 30s 602us/sample - loss: 0.0600 - accuracy: 0.9788
Epoch 7/30
50000/50000 [==============================] - 30s 601us/sample - loss: 0.0593 - accuracy: 0.9785
Epoch 8/30
50000/50000 [==============================] - 30s 598us/sample - loss: 0.0498 - accuracy: 0.9821
Epoch 9/30
50000/50000 [==============================] - 30s 599us/sample - loss: 0.0529 - accuracy: 0.9

In [ ]:
# assistive train data
model3.evaluate(X_train_assistive, y_train)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[0.012166566797296401, 0.99586]

In [ ]:
# clean test data
model3.evaluate(X_test, y_test)

[0.8451863438129426, 0.814]

In [ ]:
# assistive test data
model3.evaluate(X_test_assistive, y_test)

[0.06220840414933918, 0.9837]

In [ ]:
# adversarial test data
model3.evaluate(X_test_adv, y_test)

[14.784507040405273, 0.1109]